<a href="https://colab.research.google.com/github/paolochikalo/datascience_work/blob/master/ToxicRussiansFeatureEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install stanza
!pip install spacy-stanza
# Nessesary installations
!pip install pymorphy2==0.8
!pip install ipython-autotime
%load_ext autotime


In [4]:
from string import punctuation

import pandas as pd

from nltk.corpus import stopwords

import stanza
from spacy_stanza import StanzaLanguage
stanza.download('ru')

2020-06-16 12:59:30 INFO: Downloading default packages for language: ru (Russian)...
2020-06-16 13:00:21 INFO: Finished downloading models and saved to /root/stanza_resources.


time: 56.4 s


In [18]:
snlp = stanza.Pipeline(lang="ru", processors='tokenize,pos', use_gpu=True, pos_batch_size=1000)
nlp = StanzaLanguage(snlp)

2020-06-16 13:19:16 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2020-06-16 13:19:16 INFO: Use device: gpu
2020-06-16 13:19:16 INFO: Loading: tokenize
2020-06-16 13:19:16 INFO: Loading: pos
2020-06-16 13:19:16 INFO: Done loading processors!


time: 994 ms


In [6]:
toxic_rus = pd.read_pickle("toxic_main.pickle")
toxic_rus.head()
toxic_rus.shape

(14412, 2)

time: 89.9 ms


In [7]:
my_toxic = pd.DataFrame(toxic_rus["comment"].iloc[2000:])
my_toxic.head()
my_toxic.shape

(401, 1)

time: 5.55 ms


In [11]:
def is_noun(text):
    doc = nlp(text)   
    NOUNS = []
    for token in doc:
        if token.pos_ == "NOUN":
            NOUNS.append(str(token).lower())
    NOUNS = set(NOUNS)
    #print(doc,"NOUNS:", NOUNS)
    return " ".join(NOUNS)

time: 2.85 ms


In [22]:
def is_adjective(text):
    doc = nlp(text)    
    ADJECTIVES = []
    for token in doc:
        if token.pos_ == "ADJ":
            ADJECTIVES.append(str(token).lower())
    ADJECTIVES = set(ADJECTIVES)
    #print(doc,"ADJECTIVES:", ADJECTIVES)
    return " ".join(ADJECTIVES)

time: 2.73 ms


In [19]:
toxic_rus["nouns"] = toxic_rus["comment"].apply(is_noun)

time: 8min 25s


In [23]:
toxic_rus["adjectives"] = toxic_rus["comment"].apply(is_adjective)

time: 8min 22s


In [21]:
toxic_rus.tail()


,comment,toxic,nouns
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0,свободы скот демократии сторонник слова
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0,что-ли спасибо гоблина суку демшизы
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0,претензии мелочам обзоры артхаусятина фильмов ...
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0,общение правила обсуждения уебывайте чатиком п...
14411,До сих пор пересматриваю его видео. Орамбо кст...,0.0,контент качеством пор надежду видео орамбо кан...


time: 13.7 ms


In [24]:
pd.to_pickle(toxic_rus, "toxic_w_nouns_adj.pickle")

time: 34.7 ms


In [0]:
import dask.dataframe as dd
ddf = dd.from_pandas(toxic_rus, npartitions=8)

time: 30.7 ms


In [0]:
ddf ['nouns'] = ddf.map_partitions(is_noun,meta=(None, 'int64')).compute()

ValueError: ignored

time: 21.3 s
